In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from math import ceil, floor, trunc

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
con_lite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('max_rows',None)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
today

datetime.date(2022, 1, 6)

### Today = Last Closed Business Day

In [2]:
#today = today - timedelta(days=3)
today

datetime.date(2022, 1, 6)

### Restart and run all cells

In [3]:
def number_of_spread(x):
    maxp = x[1]
    minp = x[5]
    div = 0.0
    gap = 0.0
    spd = 0
    spd1 = 0
    spd2 = 0
    if (maxp > minp): 
        mult = 1
    else:
    	tmpp = minp
    	minp = maxp
    	maxp = tmpp
    	mult = -1
        
    gap = round(maxp - minp,2)   
    
    if (maxp > 400.00):
        div = 2.0
        boundary = div * 200
        if ((minp < boundary)):
            spd1 = trunc((boundary - minp) / 1.00)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 200.00):
        div = 1.0
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.50)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 100.00):
        div = 0.50
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.25)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 25.00):
        div = 0.25
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.10)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div)
    elif (maxp > 10.00):
        div = 0.10
        boundary = div * 100
        if (minp < boundary):
            spd1 = ceil((boundary - minp) / 0.05)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            div = 0.10            
            spd = ceil(gap / div)
    elif (maxp > 5.00):
        div = 0.05
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.02)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2           
        else:
            div = 0.05            
            spd = ceil(gap / div)
    elif (maxp > 2.00):
        div = 0.02
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.01)
            spd2 = trunc((maxp - boundary) / div)
            spd = spd1 + spd2
        else:          
            spd = (gap / div)
    else:
        div = 0.01
        boundary = div * 100        
        spd = trunc(gap / div) 
        
    return spd * mult

In [4]:
cols = 'name spd target bl qty fm_price to_price ttl min min52 css_price status amount fm_date to_date days'.split()

In [5]:
format_dict = {
               'fm_price':'{:.2f}','to_price':'{:.2f}','price':'{:.2f}',    
               'max_price':'{:.2f}','min_price':'{:.2f}',
               'target':'{:.2f}','max':'{:.2f}','max52':'{:.2f}',
               'min':'{:.2f}','min52':'{:.2f}',
               'css_price':'{:.2f}','amount':'{:,.2f}',  
               'unit_cost':'{:.2f}','cost_amt':'{:,.2f}',
               'dividend':'{:.2f}','dividend_amount':'{:,.2f}',    
               'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','cost_pct':'{:.2f}', 
               'buy_target':'{:.2f}','sell_target':'{:.2f}',
               'volume':'{:,.2f}','beta':'{:.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}','shares':'{:,}'}
pd.set_option("display.max_rows", None)

In [6]:
sql = '''
SELECT T.name,buy_target AS target, bl, T.qty,
fm_price,to_price,  ttl_spread AS ttl, A.min_price AS min, 
T.min_price AS min52, consensus_price AS css_price, buy_target * T.qty AS amount, T.status,
FM_date, to_date, days
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('B','O')
AND buy_target > 0
AND to_date = '%s'
'''
sql = sql % today
print(sql)

df_stock = pd.read_sql(sql, con_lite)
df_stock.sort_values(['name'],ascending=[True]).style.format(format_dict)


SELECT T.name,buy_target AS target, bl, T.qty,
fm_price,to_price,  ttl_spread AS ttl, A.min_price AS min, 
T.min_price AS min52, consensus_price AS css_price, buy_target * T.qty AS amount, T.status,
FM_date, to_date, days
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('B','O')
AND buy_target > 0
AND to_date = '2022-01-06'



,name,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,amount,status,fm_date,to_date,days
6,BGC,10.40,0,"9,000",10.70,10.50,-2,10.50,9.20,12.57,"93,600.00",O,2022-01-04,2022-01-06,2
7,GLOBAL,19.00,0,"7,500",20.10,19.90,-2,19.60,16.16,26.25,"142,500.00",B,2022-01-05,2022-01-06,1
3,IMH,12.00,0,"1,500",12.20,16.00,38,12.30,2.08,19.20,"18,000.00",B,2022-01-04,2022-01-06,2
8,KCE,82.50,1,"1,100",88.50,84.00,-18,83.50,44.00,102.00,"90,750.00",B,2022-01-05,2022-01-06,1
0,NOBLE,5.85,0,"21,000",6.10,6.25,3,6.10,5.90,6.71,"122,850.00",B,2021-12-28,2022-01-06,7
9,PTT,36.50,9,"3,000",39.00,38.00,-4,38.00,34.00,48.26,"109,500.00",B,2022-01-05,2022-01-06,1
10,RATCH,42.75,0,"3,000",45.25,44.50,-3,44.50,42.00,52.69,"128,250.00",B,2022-01-05,2022-01-06,1
12,ROJNA,6.50,0,"18,000",6.75,6.65,-2,6.60,4.22,8.05,"117,000.00",O,2022-01-05,2022-01-06,1
4,SIS,42.00,0,"3,000",47.75,43.75,-16,43.25,17.30,36.88,"126,000.00",B,2022-01-04,2022-01-06,2
5,STA,28.00,3,"2,500",30.00,30.50,2,29.75,25.50,45.00,"70,000.00",B,2022-01-04,2022-01-06,2


In [7]:
my_array = df_stock.to_numpy()
my_array

array([['NOBLE', 5.85, 0, 21000, 6.1, 6.25, 3, 6.1, 5.9, 6.71,
        122849.99999999999, 'B', '2021-12-28', '2022-01-06', 7],
       ['TVO', 30.0, 0, 3600, 30.0, 31.0, 4, 30.0, 29.5, 31.9, 108000.0,
        'O', '2021-12-27', '2022-01-06', 8],
       ['TSTH', 1.42, 0, 45000, 1.46, 1.5, 4, 1.46, 0.62, 1.5, 63900.0,
        'B', '2021-12-30', '2022-01-06', 5],
       ['IMH', 12.0, 0, 1500, 12.2, 16.0, 38, 12.3, 2.08, 19.2, 18000.0,
        'B', '2022-01-04', '2022-01-06', 2],
       ['SIS', 42.0, 0, 3000, 47.75, 43.75, -16, 43.25, 17.3, 36.88,
        126000.0, 'B', '2022-01-04', '2022-01-06', 2],
       ['STA', 28.0, 3, 2500, 30.0, 30.5, 2, 29.75, 25.5, 45.0, 70000.0,
        'B', '2022-01-04', '2022-01-06', 2],
       ['BGC', 10.4, 0, 9000, 10.7, 10.5, -2, 10.5, 9.2, 12.57, 93600.0,
        'O', '2022-01-04', '2022-01-06', 2],
       ['GLOBAL', 19.0, 0, 7500, 20.1, 19.9, -2, 19.6, 16.16, 26.25,
        142500.0, 'B', '2022-01-05', '2022-01-06', 1],
       ['KCE', 82.5, 1, 1100, 88.5,

In [8]:
tmp = (np.apply_along_axis(number_of_spread, axis=1, arr=my_array))
tmp

array([ -8,  -4,  -8, -40,  -7, -10,  -1,  -9,  -6,  -6,  -7, -13,  -3,
       -15])

In [9]:
df_stock['spd'] = tmp.tolist()
df_stock['spd'] = df_stock['spd'].astype(int)
df_stock.style.format(format_dict)

,name,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,amount,status,fm_date,to_date,days,spd
0,NOBLE,5.85,0,"21,000",6.10,6.25,3,6.10,5.90,6.71,"122,850.00",B,2021-12-28,2022-01-06,7,-8
1,TVO,30.00,0,"3,600",30.00,31.00,4,30.00,29.50,31.90,"108,000.00",O,2021-12-27,2022-01-06,8,-4
2,TSTH,1.42,0,"45,000",1.46,1.50,4,1.46,0.62,1.50,"63,900.00",B,2021-12-30,2022-01-06,5,-8
3,IMH,12.00,0,"1,500",12.20,16.00,38,12.30,2.08,19.20,"18,000.00",B,2022-01-04,2022-01-06,2,-40
4,SIS,42.00,0,"3,000",47.75,43.75,-16,43.25,17.30,36.88,"126,000.00",B,2022-01-04,2022-01-06,2,-7
5,STA,28.00,3,"2,500",30.00,30.50,2,29.75,25.50,45.00,"70,000.00",B,2022-01-04,2022-01-06,2,-10
6,BGC,10.40,0,"9,000",10.70,10.50,-2,10.50,9.20,12.57,"93,600.00",O,2022-01-04,2022-01-06,2,-1
7,GLOBAL,19.00,0,"7,500",20.10,19.90,-2,19.60,16.16,26.25,"142,500.00",B,2022-01-05,2022-01-06,1,-9
8,KCE,82.50,1,"1,100",88.50,84.00,-18,83.50,44.00,102.00,"90,750.00",B,2022-01-05,2022-01-06,1,-6
9,PTT,36.50,9,"3,000",39.00,38.00,-4,38.00,34.00,48.26,"109,500.00",B,2022-01-05,2022-01-06,1,-6


In [10]:
df_stock[cols].style.format(format_dict)

,name,spd,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,status,amount,fm_date,to_date,days
0,NOBLE,-8,5.85,0,"21,000",6.10,6.25,3,6.10,5.90,6.71,B,"122,850.00",2021-12-28,2022-01-06,7
1,TVO,-4,30.00,0,"3,600",30.00,31.00,4,30.00,29.50,31.90,O,"108,000.00",2021-12-27,2022-01-06,8
2,TSTH,-8,1.42,0,"45,000",1.46,1.50,4,1.46,0.62,1.50,B,"63,900.00",2021-12-30,2022-01-06,5
3,IMH,-40,12.00,0,"1,500",12.20,16.00,38,12.30,2.08,19.20,B,"18,000.00",2022-01-04,2022-01-06,2
4,SIS,-7,42.00,0,"3,000",47.75,43.75,-16,43.25,17.30,36.88,B,"126,000.00",2022-01-04,2022-01-06,2
5,STA,-10,28.00,3,"2,500",30.00,30.50,2,29.75,25.50,45.00,B,"70,000.00",2022-01-04,2022-01-06,2
6,BGC,-1,10.40,0,"9,000",10.70,10.50,-2,10.50,9.20,12.57,O,"93,600.00",2022-01-04,2022-01-06,2
7,GLOBAL,-9,19.00,0,"7,500",20.10,19.90,-2,19.60,16.16,26.25,B,"142,500.00",2022-01-05,2022-01-06,1
8,KCE,-6,82.50,1,"1,100",88.50,84.00,-18,83.50,44.00,102.00,B,"90,750.00",2022-01-05,2022-01-06,1
9,PTT,-6,36.50,9,"3,000",39.00,38.00,-4,38.00,34.00,48.26,B,"109,500.00",2022-01-05,2022-01-06,1


In [11]:
buy_alert = df_stock[cols]
buy_alert.set_index(['name'],inplace=True)
buy_alert.sort_values(['spd'],ascending=[False]).style.format(format_dict)

,spd,target,bl,qty,fm_price,to_price,ttl,min,min52,css_price,status,amount,fm_date,to_date,days
name,,,,,,,,,,,,,,,
BGC,-1,10.40,0,"9,000",10.70,10.50,-2,10.50,9.20,12.57,O,"93,600.00",2022-01-04,2022-01-06,2
ROJNA,-3,6.50,0,"18,000",6.75,6.65,-2,6.60,4.22,8.05,O,"117,000.00",2022-01-05,2022-01-06,1
TVO,-4,30.00,0,"3,600",30.00,31.00,4,30.00,29.50,31.90,O,"108,000.00",2021-12-27,2022-01-06,8
KCE,-6,82.50,1,"1,100",88.50,84.00,-18,83.50,44.00,102.00,B,"90,750.00",2022-01-05,2022-01-06,1
PTT,-6,36.50,9,"3,000",39.00,38.00,-4,38.00,34.00,48.26,B,"109,500.00",2022-01-05,2022-01-06,1
SIS,-7,42.00,0,"3,000",47.75,43.75,-16,43.25,17.30,36.88,B,"126,000.00",2022-01-04,2022-01-06,2
RATCH,-7,42.75,0,"3,000",45.25,44.50,-3,44.50,42.00,52.69,B,"128,250.00",2022-01-05,2022-01-06,1
NOBLE,-8,5.85,0,"21,000",6.10,6.25,3,6.10,5.90,6.71,B,"122,850.00",2021-12-28,2022-01-06,7
TSTH,-8,1.42,0,"45,000",1.46,1.50,4,1.46,0.62,1.50,B,"63,900.00",2021-12-30,2022-01-06,5


In [12]:
file_name = 'buy-alert.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name

buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(output_file)
buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(data_file)
buy_alert.sort_values(['spd','name'],ascending=[False,True]).to_csv(box_file)